<a href="https://colab.research.google.com/github/jaidvedant/BML-LCA/blob/main/exp_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/data/iris.csv'

column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

df = pd.read_csv(file_path, header = None, names = column_names)

print(df.head())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df

x = df[['sepal_length','sepal_width','petal_length','petal_width']]
y = df['class']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("Training size:", x_train.shape)
print("Testing size:", x_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   sepal_length  sepal_width  petal_length  petal_width    class
0  sepal.length  sepal.width  petal.length  petal.width  variety
1           5.1          3.5           1.4           .2   Setosa
2           4.9            3           1.4           .2   Setosa
3           4.7          3.2           1.3           .2   Setosa
4           4.6          3.1           1.5           .2   Setosa
Training size: (120, 4)
Testing size: (31, 4)


In [15]:
from collections import Counter
import numpy as np
import pandas as pd # Import pandas for to_numeric and dropna

# Manual KNN function
def knn_predict(x_train_in, y_train_in, X_test_in, k):
    predictions = []

    x_train_numeric = x_train_in.apply(pd.to_numeric, errors='coerce')
    X_test_numeric = X_test_in.apply(pd.to_numeric, errors='coerce')

    x_train_cleaned = x_train_numeric.dropna()
    y_train_cleaned = y_train_in.loc[x_train_cleaned.index]


    X_test_cleaned = X_test_numeric.dropna()

    for test_sample in X_test_cleaned.values:
        # Compute Euclidean distances
        distances = np.sqrt(np.sum((x_train_cleaned.values - test_sample) ** 2, axis=1))

        # Get indices of k nearest neighbors
        k_neighbors = np.argsort(distances)[:k]

        # Get labels of k nearest neighbors
        k_labels = [y_train_cleaned.iloc[i] for i in k_neighbors]

        # Find most common class label
        most_common = Counter(k_labels).most_common(1)[0][0]
        predictions.append(most_common)

    return np.array(predictions), X_test_cleaned.index

# Set k value
k = 5
# Make copies to ensure original x_train and x_test are not altered by the function if they are used elsewhere.
x_train_copy = x_train.copy()
y_train_copy = y_train.copy()
x_test_copy = x_test.copy()

y_pred_manual, cleaned_test_indices = knn_predict(x_train_copy, y_train_copy, x_test_copy, k)

# Calculate accuracy
# y_test needs to be aligned with the cleaned X_test
y_test_aligned = y_test.loc[cleaned_test_indices]

# Only compare predictions for the samples that were successfully processed
accuracy_manual = np.mean(y_pred_manual == y_test_aligned.values)

print(f"Manual KNN Accuracy with k={k}: {accuracy_manual:.2%}")



Manual KNN Accuracy with k=5: 96.77%


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Initialize KNN model
model = KNeighborsClassifier(n_neighbors=20)

x_train_numeric = x_train.apply(pd.to_numeric, errors='coerce')

# Identify rows that contain any NaN values introduced by the coercion (these were the string rows).
rows_with_strings = x_train_numeric.isnull().any(axis=1)

x_train_cleaned = x_train_numeric[~rows_with_strings]
y_train_cleaned = y_train[~rows_with_strings]

model.fit(x_train_cleaned, y_train_cleaned)

x_test_numeric = x_test.apply(pd.to_numeric, errors='coerce')
x_test_cleaned = x_test_numeric.dropna()

y_test_cleaned = y_test.loc[x_test_cleaned.index]

y_pred = model.predict(x_test_cleaned)

print("Testing Score:", model.score(x_test_cleaned, y_test_cleaned))
print("Training Score:", model.score(x_train_cleaned, y_train_cleaned))
print("Accuracy Score:", accuracy_score(y_test_cleaned, y_pred))

Testing Score: 0.9354838709677419
Training Score: 0.9747899159663865
Accuracy Score: 0.9354838709677419
